# **Requirements**

In [131]:
import pandas as pd
import os
import ipaddress

from urllib.parse import urlparse, urlsplit, parse_qs
import whois
from datetime import datetime
import socket
import pydig
import certifi
import urllib3
from cymruwhois import Client

import tldextract
import dns
import dns.name
import dns.query
import dns.resolver

import re
import requests
from bs4 import BeautifulSoup

import validators

Import URL List

1. Phishing

In [119]:
phish_data = pd.read_csv("Dataset/Phishtank/phishtank_0107.csv")
phish_data.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7572770,http://acessossl110.ddns.net/cadastro.php?codi...,http://www.phishtank.com/phish_detail.php?phis...,2022-07-01T15:06:54+00:00,yes,2022-07-01T15:21:25+00:00,yes,Other
1,7572768,http://www.vivsvesi.vsccacsei.lxqslk.top/ic6oX...,http://www.phishtank.com/phish_detail.php?phis...,2022-07-01T15:06:44+00:00,yes,2022-07-01T15:21:25+00:00,yes,Other
2,7572767,http://a0691990.xsph.ru/,http://www.phishtank.com/phish_detail.php?phis...,2022-07-01T15:06:17+00:00,yes,2022-07-01T15:21:25+00:00,yes,Other
3,7572756,https://agosttoo-9948200.azurewebsites.net/ini...,http://www.phishtank.com/phish_detail.php?phis...,2022-07-01T15:04:45+00:00,yes,2022-07-01T15:21:25+00:00,yes,Other
4,7572754,https://aquaairfl.com/s/itau.com.cz/p/?d41d8cd...,http://www.phishtank.com/phish_detail.php?phis...,2022-07-01T15:04:28+00:00,yes,2022-07-01T15:21:25+00:00,yes,Other


In [120]:
legit_data = pd.read_csv("Dataset/UNB/Benign_list_big_final.csv")
legit_data.columns = ['URLs']
legit_data.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [121]:
legiturl = legit_data.sample(n = 200, random_state = 12).copy()
legiturl = legiturl.reset_index(drop=True)
legiturl.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [122]:
legiturl.to_csv('valid_1.csv', index= False)

## **Table 1**
### **Dataset attributes based on URL.**

### *Element*

In [123]:
def qty_dot(url):
    return url.count('.')

In [124]:
def qty_hyphen(url):
    return url.count("-")

In [125]:
def qty_underline(url):
    return url.count('_')

In [126]:
def qty_slash(url):
    return url.count('/')

In [127]:
def qty_questionmark(url):
    return url.count('?')

In [128]:
def qty_equal(url):
    return url.count('=')

In [129]:
def qty_at(url):
    return url.count('@')

In [130]:
def qty_and(url):
    return url.count('&')

In [131]:
def qty_exclamation(url):
    return url.count('!')

In [132]:
def qty_space(url):
    return url.count(' ')

In [133]:
def qty_tilde(url):
    return url.count('~')

In [134]:
def qty_comma(url):
    return url.count(',')

In [135]:
def qty_plus(url):
    return url.count('+')

In [136]:
def qty_asterisk(url):
    return url.count('*')

In [137]:
def qty_hashtag(url):
    return url.count('#')

In [138]:
def qty_dollar(url):
    return url.count('$')

In [139]:
def qty_percent(url):
    return url.count('%')

### *The rest of table 1*

In [140]:
def qty_tld_url(url):
    tld_url = tldextract.extract(url).suffix
    return len(tld_url)

In [141]:
def length_url(url):
    return len(url)

In [142]:
emailRegex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
 
def email_in_url(url):
    return 1 if (re.findall(emailRegex, url)) else 0

## **Table 2**
### **Dataset attributes based on domain URL.**

In [143]:
def qty_vowels_domain(url):
  vowels = ['a', 'e', 'i', 'o', 'u']
  return len([i for i in url if i in vowels])

def domain_length(url):
  return len(url)

def domain_in_ip(url):
  ipPattern = re.compile('\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
  try:
      mo = ipPattern.search(url).group()
      if ipaddress.ip_address(mo):
          ip = 1
  except Exception:
      ip = 0
  return ip

def server_client_domain(url):
  keyword = r"server|client"
  return 1 if (re.search(keyword,url)) else 0

## **Table 3**
### **Dataset attributes based on URL directory.**

## **Table 4**
### **Dataset attributes based on URL file name.**

## **Table 5**
### **Dataset attributes based on URL parameters.**

In [144]:
def tld_present_params(url):
    return 1 if tldextract.extract(url).suffix else 0

In [145]:
def qty_params(url):
    return len(url.split("&")) if "&" in url else 1

## **Table 6**
### **Dataset attributes based on resolving URL and external services.**

### *6.1. Domain lookup time response*

In [146]:
def time_response(url):
    try:
        r = requests.get(url, timeout=6)
        r.raise_for_status()
        return round(r.elapsed.total_seconds(),7)
    except Exception:
        return 0

# respTime = r.elapsed.total_seconds()


In [147]:
# FOR TESTING
# url = 'https://expdfired.badssl.com'
# url = 'https://socs1.binus.ac.id/proposalskripsi/auth/login'
url = 'https://www.knowledgehut.com/tutorials/python-tutorial/python-lists-tuples'
time_response(url)

0.090504

### *6.2. Domain has SPF*

In [148]:
def domain_spf(url):
  try:
    test_spf = dns.resolver.resolve(url , 'TXT')
    for dns_data in test_spf:
      if 'spf1' in str(dns_data):
        return 1
  except Exception:
    return 0

In [149]:
# FOR TESTING
# url = 'https://expdfired.badssl.com'
url = 'https://socds1.binus.ac.id/proposalskripsi/auth/login'
# url = 'https://www.knowledgehut.com/tutorials/python-tutorial/python-lists-tuples'
domain_spf(url)

0

### *6.3. ASN*

In [150]:
def asn_ip(url):
    try:
        domainUrl = urlparse(url).hostname
        ip = socket.gethostbyname(domainUrl)
        c = Client()
        r = c.lookup(ip)
        return r.asn
    except Exception:
        return 0


### *6.4. Domain activation time*

In [151]:
def time_domain_activation(url):
  today = datetime.now()

  try:
    creation_date = whois.whois(urlparse(url).netloc).creation_date
  except Exception:
    return 0

  if isinstance(creation_date,list):
    creation_date = creation_date[0]

  if isinstance(creation_date,str):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      # expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except Exception:
      return 0
  if (creation_date is None) or type(creation_date) is list:
    return 0
  return abs((creation_date - today).days)

In [152]:
# FOR TESTING
# url = 'https://stackoverflow.com/questions/6197572/change-statusbar-onmouseover-in-a-div'
url = 'https://securiti.ai/privaci/universal-consent-preference-management/'
time_domain_activation(url)
# time_domain_expiration(url)

0

### *6.5. Domain expiration time*

In [153]:
def time_domain_expiration(url):
  today = datetime.now()

  try:
    expiration_date = whois.whois(urlparse(url).netloc).expiration_date
  except Exception:
    return 0

  if isinstance(expiration_date,list):
    expiration_date = expiration_date[0]

  if isinstance(expiration_date,list):
    try:
        expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        return expiration_date
    except Exception:
      return 0
  if (expiration_date is None) or type(expiration_date) is list:
    return 0
  return abs((today - expiration_date).days)


In [154]:
# FOR TESTING
# url = 'https://stackoverflow.com/questions/6197572/change-statusbar-onmouseover-in-a-div'
url = 'https://www.youtube.com'
# time_domain_activation(url)
time_domain_expiration(url)

226

### *6.6. Number of resolved IPs*

In [155]:
def qty_ip_resolved(url):
    try:
        domain = whois.whois(urlparse(url).netloc).domain_name
    except Exception:
        return 0

    if domain is None:
        return 0

    if isinstance(domain,list):
        try:
            domain = domain[0]
        except Exception:
            return 0
    try:
        ipAddress = socket.gethostbyname_ex(domain)
        return len(ipAddress[-1])
    except Exception:
        return 0

In [156]:
# FOR TESTING
# url = 'https://socs1.binus.ac.id/proposalskripsi/auth/login'
url = 'https://www.knowledgehut.com/tutorials/python-tutorial/python-lists-tuples'


qty_ip_resolved(url)

4

### *6.7. Number of resolved NS*

In [157]:
def qty_nameservers(url):
    try:
        nameServer = whois.whois(urlparse(url).netloc).name_servers
        # nameServer = 
    except Exception:
        return 0

    if nameServer is None:
        return 0
    if isinstance(nameServer,str):
        return 1

    return len(nameServer)

In [158]:
# FOR TESTING
url = 'https://www.knowledgehdut.com/tutorials/python-tutorial/python-lists-tuples'
# url = 'https://myclass.apps.binus.ac.id'
# domain = whois.whois(urlparse(url).netloc).domain_name
qty_nameservers(url)

0

### *6.8. Number of MX servers*

In [159]:
def qty_mx_servers(url):
    try:
        domain = whois.whois(urlparse(url).netloc).domain_name
        try:
            return len(dns.resolver.resolve(domain, 'MX'))
        except Exception:
            return 0
    except Exception:
        return 0


In [160]:
# FOR TESTING
# url = 'https://socs1.binus.ac.id/proposalskripsi/auth/login'
url = 'https://www.knowledgsdehut.com/tutorials/python-tutorial/python-lists-tuples'
qty_mx_servers(url)

0

### *6.9. Time-To-Live*

In [161]:
def ttl_hostname(url):
    my_resolver = dns.resolver.Resolver()
    my_resolver.nameservers = ['1.1.1.1']

    domain = urlparse(url).netloc
    # print (ns2.binus.ac.id.)
    try:
        ns = get_authoritative_nameserver(domain)
        # print(ns)
        # answer = my_resolver.query('ns1-05.azure-dns.com.')
        # my_resolver.query('ns2.binus.ac.id.').rrset.ttl
        return my_resolver.query(ns).rrset.ttl
    except Exception:
        return 0

def get_authoritative_nameserver(domain, log=lambda msg: None):
    n = dns.name.from_text(domain)

    depth = 2
    default = dns.resolver.get_default_resolver()
    nameserver = default.nameservers[0]

    last = False
    while not last:
        s = n.split(depth)

        last = s[0].to_unicode() == u'@'
        sub = s[1]

        log('Looking up %s on %s' % (sub, nameserver))
        query = dns.message.make_query(sub, dns.rdatatype.NS)
        response = dns.query.udp(query, nameserver)

        rcode = response.rcode()
        if rcode != dns.rcode.NOERROR:
            if rcode == dns.rcode.NXDOMAIN:
                raise Exception('%s does not exist.' % sub)
            else:
                raise Exception('Error %s' % dns.rcode.to_text(rcode))

        rrset = None
        if len(response.authority) > 0:
            rrset = response.authority[0]
        else:
            rrset = response.answer[0]

        rr = rrset[0]
        if rr.rdtype == dns.rdatatype.SOA:
            log('Same server is authoritative for %s' % sub)
        else:
            authority = rr.target
            log('%s is authoritative for %s' % (authority, sub))
            nameserver = default.query(authority).rrset[0].to_text()

        depth += 1

    return authority


import sys

def log(msg):
    print (msg)

In [162]:
# FOR TESTING
# url = 'https://socs1.binus.ac.id/proposalskripsi/auth/login'
url = 'https://www.knowledgehut.com/tutorials/python-tutorial/python-lists-tuples'
# url = 'https://stackoverflow.com/questions/26930572/count-number-of-redirects-in-scrapy'
domain = urlparse(url).netloc

ttl_hostname(domain)

0

### *6.10. Has valid TLS/SSL certificate*

In [163]:
def tls_ssl_certificate(url):
    try:
        r = requests.get(url)
        return 1
    except (requests.exceptions.SSLError):
        # print(traceback.format_exc())
        return 0


In [164]:
# FOR TESTING
# url = 'https://expdfired.badssl.com'
# url = 'https://socs1.binus.ac.id/proposalskripsi/auth/login'
url = 'https://www.knowledgehut.com/tutorials/python-tutorial/python-lists-tuples'
tls_ssl_certificate(url)

1

### *6.11. Number of redirects*

In [165]:
def qty_redirects(url):
    try:
        response = requests.get(url, allow_redirects = True)
        # print(response.history)
        return sum(1 for _ in response.history)
        
    except requests.exceptions.SSLError:
        # log.error(
        #     'Certificate did not match expected hostname: %s. '
        # )
        return 0



In [166]:
# FOR TESTING

# url = 'https://socs1.binus.ac.id/proposalskripsi/auth/login'
# url = 'https://www.knowledgehut.com/tutorials/python-tutorial/python-lists-tuples'
url = 'https://facebok.com/'
# url = 'http://httpbin.org/redirect/3'
qty_redirects(url)

0

### *6.12. Is URL/Domain indexed on Google*

In [167]:
def indexed(url):
  

  google = f"https://www.google.com/search?q=site:{url}&hl=en"
  response = requests.get(google, cookies={"CONSENT": "YES+1"})
  soup = BeautifulSoup(response.content, "html.parser")
  not_indexed = re.compile("did not match any documents")

  return 0 if soup(text=not_indexed) else 1

def url_google_index(url):
  return indexed(url)

def domain_google_index(url):
  domainURL = urlparse(url).hostname
  return indexed(domainURL)

In [168]:
# FOR TESTING

url = 'https://socs1.binus.ac.id/proposalskripsi/auth/login'
# url = 'https://www.knowsdfsdfledgdsfsdfehut.com/tutorials/python-tutorial/python-lists-tuples'
# url = 'https://facebook.com/'
# url = 'http://httpbin.org/redirect/3'
url_google_index(url)
# domain_google_index(url)

0

### *6.13. Is URL shortened*

In [169]:
# Is URL shortened

# !pip install urlunshort3
# import urlunshort3
# is_shortened(url)

short_url = re.compile(r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
            r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
            r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
            r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
            r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
            r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
            r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
            r"tr\.im|link\.zip\.net")
# def url_shortened(url):
#     match = re.search(short_url,url)
#     url_shortened = 1 if match else 0
#     url_shortened

def url_shortened(url):
    domain = urlparse(url).netloc
    return 1 if short_url.match(domain) else 0

## **Testing**

In [174]:
def featureExtract(url):

  features = []

  features.append(qty_dot(url))
  features.append(qty_hyphen(url))
  features.append(qty_underline(url))
  features.append(qty_slash(url))
  features.append(qty_questionmark(url))
  features.append(qty_equal(url))
  features.append(qty_at(url))
  features.append(qty_and(url))
  features.append(qty_exclamation(url))
  features.append(qty_space(url))
  features.append(qty_tilde(url))
  features.append(qty_comma(url))
  features.append(qty_plus(url))
  features.append(qty_asterisk(url))
  features.append(qty_hashtag(url))
  features.append(qty_dollar(url))
  features.append(qty_percent(url))
  features.append(qty_tld_url(url))
  features.append(length_url(url))

  try:
    domainUrl = urlparse(url).hostname
  except Exception:
    domainUrl = 0
    
  features.append(0 if domainUrl == 0 else qty_dot(domainUrl))
  features.append(0 if domainUrl == 0 else qty_hyphen(domainUrl))
  features.append(0 if domainUrl == 0 else qty_underline(domainUrl))
  features.append(0 if domainUrl == 0 else qty_slash(domainUrl))
  features.append(0 if domainUrl == 0 else qty_questionmark(domainUrl))
  features.append(0 if domainUrl == 0 else qty_equal(domainUrl))
  features.append(0 if domainUrl == 0 else qty_at(domainUrl))
  features.append(0 if domainUrl == 0 else qty_and(domainUrl))
  features.append(0 if domainUrl == 0 else qty_exclamation(domainUrl))
  features.append(0 if domainUrl == 0 else qty_space(domainUrl))
  features.append(0 if domainUrl == 0 else qty_tilde(domainUrl))
  features.append(0 if domainUrl == 0 else qty_comma(domainUrl))
  features.append(0 if domainUrl == 0 else qty_plus(domainUrl))
  features.append(0 if domainUrl == 0 else qty_asterisk(domainUrl))
  features.append(0 if domainUrl == 0 else qty_hashtag(domainUrl))
  features.append(0 if domainUrl == 0 else qty_dollar(domainUrl))
  features.append(0 if domainUrl == 0 else qty_percent(domainUrl))
  features.append(0 if domainUrl == 0 else qty_vowels_domain(domainUrl))
  features.append(0 if domainUrl == 0 else domain_length(domainUrl))
  features.append(0 if domainUrl == 0 else domain_in_ip(domainUrl))
  features.append(0 if domainUrl == 0 else server_client_domain(domainUrl))

  try:
    directoryUrl = os.path.dirname(urlparse(url).path)
    if not directoryUrl:
      directoryUrl = 0
  except Exception:
    directoryUrl = 0

  features.append(0 if directoryUrl == 0 else qty_dot(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_hyphen(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_underline(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_slash(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_questionmark(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_equal(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_at(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_and(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_exclamation(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_space(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_tilde(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_comma(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_plus(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_asterisk(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_hashtag(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_dollar(directoryUrl))
  features.append(0 if directoryUrl == 0 else qty_percent(directoryUrl))
  features.append(0 if directoryUrl == 0 else domain_length(directoryUrl))

  try:
    filenameUrl = os.path.basename(urlparse(url).path)
    if not filenameUrl:
      filenameUrl = 0
  except Exception:
    filenameUrl = 0

  features.append(0 if filenameUrl == 0 else qty_dot(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_hyphen(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_underline(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_slash(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_questionmark(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_equal(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_at(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_and(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_exclamation(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_space(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_tilde(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_comma(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_plus(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_asterisk(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_hashtag(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_dollar(filenameUrl))
  features.append(0 if filenameUrl == 0 else qty_percent(filenameUrl))
  features.append(0 if filenameUrl == 0 else domain_length(filenameUrl))

  try:
    parameterUrl = urlparse(url).query
    if not parameterUrl:
      parameterUrl = 0
  except Exception:
    parameterUrl = 0

  features.append(0 if parameterUrl == 0 else qty_dot(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_hyphen(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_underline(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_slash(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_questionmark(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_equal(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_at(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_and(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_exclamation(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_space(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_tilde(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_comma(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_plus(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_asterisk(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_hashtag(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_dollar(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_percent(parameterUrl))
  features.append(0 if parameterUrl == 0 else domain_length(parameterUrl))
  features.append(0 if parameterUrl == 0 else tld_present_params(parameterUrl))
  features.append(0 if parameterUrl == 0 else qty_params(parameterUrl))
  features.append(email_in_url(url))


  features.append(time_response(url))
  features.append(domain_spf(url))
  features.append(asn_ip(url))
  features.append(time_domain_activation(url))
  features.append(time_domain_expiration(url))
  features.append(qty_ip_resolved(url))
  features.append(qty_nameservers(url))
  features.append(qty_mx_servers(url))
  features.append(0)
  features.append(tls_ssl_certificate(url))
  features.append(qty_redirects(url))
  features.append(url_google_index(url))
  features.append(domain_google_index(url))
  features.append(url_shortened(url))


  



  print (features)

from concurrent.futures import ThreadPoolExecutor


list_domain = ['http://www.mascesceard.mcasord.mltuey.top/ic6oXx7P3s/page1.php',
              'https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html'
              ]


with ThreadPoolExecutor() as executor:
    executor.map(featureExtract, list_domain)


[5, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 62, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 34, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.486414, 0, '36352', 222, 144, 0, 2, 0, 0, 1, 0, 0, 0, 0]
[4, 1, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 94, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 16, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.081132, 0, '54113', 0, 0, 0, 3, 0, 0, 1, 0, 1, 1, 0]


In [ ]:
regexUrl = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

# print(re.match(regexUrl, "http://www.example.com") is not None) # True
# print(re.match(regexUrl, "example.com") is not None)   


def urlValid(url):
    if re.match(regexUrl, url) is not None:
        return featureExtract('https://www.knowledgehut.com/tutorials/python-tutorial/python-lists-tuples')

        # valid=validators.url(url)
        # if valid==True:
        #     print("Url is valid")
        # else:
        #     print("Invalid url")
    else:
        return 0

In [ ]:
# urlValid('https://pand$$$$as@asd.py~da,-ta.org?/doc&!%++****#2,,00@s/user_guide/merging.html')
urlValid('https://www.knowledgehut.com/tutorials/python-tutorial/python-lists-tuples')
# urlValid('https://pand$$$$as@asd.py~da,-t a.org?/doc&!%++****#2,,00@s/user_guide/merging.html')
# urlValid('https://www.example.com/some_path?some_key=some_valuesdfs&asdasd=asd.com')

[0.034778, 0, '16509', 4001, 1479, 4, 4, 0, 1, 0, 1, 1, 0]

In [ ]:
len(urlValid('https://www.knowledgehut.com/tutorials/python-tutorial/python-lists-tuples'))

13

generate dataset

In [154]:
def whoisDomain(url):
    try:
        domain = whois.whois(urlparse(url).netloc)

    except Exception:
        return 0

    if domain is None:
        return 0

    if isinstance(domain,list):
        try:
            domain = domain[0]
        except Exception:
            return 0
    return domain

In [148]:
def city(url):
    return domain.address

In [164]:
def WhoisExp(url):
    try:
        expiration_date = whois.whois(urlparse(url).netloc).expiration_date
    except Exception:
        return 0

    try:
        return expiration_date.strftime("%m/%d/%Y, %H:%M:%S")
    except Exception:
        return 0
    

In [165]:
url = "https://stackoverflow.com/questions/72910205/there-is-a-way-to-publish-and-consume-message-from-django-to-react"
exp(url)

'02/02/2023, 11:59:59'

In [138]:
url = "https://stackoverflow.com/questions/72910205/there-is-a-way-to-publish-and-consume-message-from-django-to-react"
whoisDomain(url)


{'domain_name': 'STACKOVERFLOW.COM',
 'registrar': 'Name.com, Inc.',
 'whois_server': 'whois.name.com',
 'referral_url': None,
 'updated_date': datetime.datetime(2022, 6, 10, 10, 45, 40),
 'creation_date': datetime.datetime(2003, 12, 26, 19, 18, 7),
 'expiration_date': datetime.datetime(2023, 2, 2, 11, 59, 59),
 'name_servers': ['NS-1033.AWSDNS-01.ORG',
  'NS-358.AWSDNS-44.COM',
  'NS-CLOUD-E1.GOOGLEDOMAINS.COM',
  'NS-CLOUD-E2.GOOGLEDOMAINS.COM',
  'ns-1033.awsdns-01.org',
  'ns-358.awsdns-44.com',
  'ns-cloud-e1.googledomains.com',
  'ns-cloud-e2.googledomains.com'],
 'status': ['ok https://icann.org/epp#ok', 'ok https://www.icann.org/epp#ok'],
 'emails': 'abuse@name.com',
 'dnssec': ['unsigned', 'unSigned'],
 'name': 'Sysadmin Team',
 'org': 'Stack Exchange, Inc.',
 'address': '110 William St , Floor 28',
 'city': 'New York',
 'state': 'NY',
 'zipcode': '10038',
 'country': 'US'}

In [24]:
del dict_whois['creation_date']


In [29]:
dict_whois.status[0]

'clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited'

In [10]:
import json 

In [12]:
json_object = json.dumps(dict_whois, indent = 10) 
print(json_object)

TypeError: Object of type datetime is not JSON serializable

testing for features

In [ ]:
# import sys
# import time
# from PyQt4.QtCore import *
# from PyQt4.QtGui import *
# from PyQt4.QtWebKit import *

# class Screenshot(QWebView):
#     def __init__(self):
#         self.app = QApplication(sys.argv)
#         QWebView.__init__(self)
#         self._loaded = False
#         self.loadFinished.connect(self._loadFinished)

#     def capture(self, url, output_file):
#         self.load(QUrl(url))
#         self.wait_load()
#         # set to webpage size
#         frame = self.page().mainFrame()
#         self.page().setViewportSize(frame.contentsSize())
#         # render image
#         image = QImage(self.page().viewportSize(), QImage.Format_ARGB32)
#         painter = QPainter(image)
#         frame.render(painter)
#         painter.end()
#         print ('saving', output_file)
#         image.save(output_file)

#     def wait_load(self, delay=0):
#         # process app events until page loaded
#         while not self._loaded:
#             self.app.processEvents()
#             time.sleep(delay)
#         self._loaded = False

#     def _loadFinished(self, result):
#         self._loaded = True

# s = Screenshot()
# s.capture('http://webscraping.com', 'website.png')
# s.capture('http://webscraping.com/blog', 'blog.png')

ModuleNotFoundError: No module named 'PyQt4'

In [ ]:

from selenium import webdriver

image_name = f"screenshot_{str(datetime.now())}"
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--test-type")
# options.add_argument("--headless")
# options.binary_location = "/usr/bin/chromium"
driver = webdriver.Chrome(options=options)

driver.get('http://www.gogle.com')
driver.save_screenshot(f"screenshot/{image_name}.png")

driver.close()



In [ ]:
os.path.basename("screenshot/screenshot_2022-07-02 21:52:19.901500.png")

'screenshot_2022-07-02 21:52:19.901500.png'

In [4]:
def report_url(url, email):
    r = requests.post('https://report.netcraft.com/api/v2/report/urls', json={
        "email": email,
        "urls": [url],
    })

    return (r.json())



In [5]:
url = "https://megaparseonalse.com/female-escorts/11887"
email = "wiehong@gmail.com"
report_status4 = report_url(url, email)
report_status4

{'message': 'Successfully reported',
 'uuid': 'dkKnpbz1YQANt5XMlNIYG9x9AttIklbQ'}

In [7]:
type(report_status4)

dict

In [13]:
if re.search("Successfully", report_status4['message']):
    print(1)
else:
  print(0)


1


In [ ]:
report_status2

{'details': [{'input': 'wiehon.com',
   'message': 'Does not match email format.',
   'path': '/body/email'}],
 'error': 'Bad Request',
 'status': 400}

In [ ]:
report_status

{'message': 'Successfully reported',
 'uuid': '44D5KHj9i3vyXwJWCABTJXoxJDGls2UF'}

In [ ]:
txt = "The rain in Spain"
x = re.search("ai", txt)

if x:
  print("YES! We have a match!")
else:
  print("No match")

# print(x)


YES! We have a match!
<re.Match object; span=(5, 7), match='ai'>


In [45]:
from random import randint, randrange

# randint(100, 999)     # randint is inclusive at both ends
randrange(10000, 99999)  # randrange is exclusive at the stop


76176

In [130]:
import random
id = randrange(10000000, 99999999)

51895334

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import requests
from requests.exceptions import ConnectionError

def validate_existence(domain):
    try:
        response = requests.get(domain, timeout=10)
    except ConnectionError:
        print(f'Domain {domain} [---]')
        # print(f'[---]')


# url = 'https://securiti.ai/privaci/universal-consent-preference-management/'
# url = 'http://www.mascsdesceard.mcasord.mltuey.top/ic6oXx7P3s/page1.php'
# url = 'http://diply.com/ilovemydog/puppies-are-little-troublemakers-in-a-cute-cloak/119633'
list_domain = ["http://graphicriver.net/search?date=this-month&length_max=&length_min=&price_max=&price_min=&rating_min=&sales=&sort=sales&term=&view=list",
"http://ecnavi.jp/redirect/?url=http://www.cross-a.net/x.php?id=1845_3212_22061_26563&m=1004&pid=%user_id%",
"https://hubpages.com/signin?explain=follow+Hubs&url=%2Fhub%2FComfort-Theories-of-Religion",
"http://extratorrent.cc/torrent/4190536/AOMEI+Backupper+Technician+%2B+Server+Edition+2.8.0+%2B+Patch+%2B+Key+%2B+100%25+Working.html",
"http://icicibank.com/Personal-Banking/offers/offer-detail.page?id=offer-ezeego-domestic-airtravel-20141407112611060",
"http://nypost.com/2015/05/07/us-indifference-leaves-saudis-partnering-with-terrorists/",
"http://kienthuc.net.vn/diem-thi/diem-chuan-dh-cong-nghe-giao-thong-van-tai-nam-2014-482407.html",
"http://thenextweb.com/in/2015/04/16/india-wants-a-neutral-web-and-facebooks-internet-org-cant-be-a-part-of-it/gtm.js",
"http://tobogo.net/cdsb/board.php?board=greet&bm=view&no=5716&category=&auth=&page=1&search=&keyword=&recom=",
"http://akhbarelyom.com/news/newdetails/411395/1/%D9%85%D8%AD%D8%A7%D9%81%D8%B8-%D8%A7%D9%84%D8%A8%D8%AD%D9%8A%D8%B1.html",
"http://tunein.com/radio/Carolina-Mudcats-at-Myrtle-Beach-Pelicans-May-13-2015-p729095/15480783/ca-pub-1542925551861702/ProgramTopBanner",
"https://tune.pk/video/6046458/canelo-vs-kirkland-highlights-hbo-world-championship-boxing",
"http://sfglobe.com/2015/05/01/six-baltimore-police-officers-charged-in-freddie-grays-death/?src=home_feed",
"http://mic.com/articles/107372/49-tremendous-things-florida-men-accomplished-this-year",
"http://thenextweb.com/apps/2015/05/07/flickr-launches-redesigned-mobile-apps-encourages-you-to-upload-your-entire-camera-roll/gtm.js",
"http://couchtuner.eu.com/2013/05/anger-management-s2-e17-charlie-lets-kate-take-charge.html",
"http://olx.in/hi/item/nokia-5800-express-music-perfect-working-condition-body-average-cond-IDTIzUp.html",
"http://venturebeat.com/2015/05/08/yelp-sale-rumor-reminds-us-that-social-is-terrible-as-a-business-even-if-it-did-change-the-web/",
"http://allegro.pl/listing/listing.php?id=122233&order=m&string=%7Bstring%7D&bmatch=seng-ps-mp-p-sm-isqm-2-e-0402",
"http://allegro.pl/listing/listing.php?id=20782&order=m&p=2&string=%7Bstring%7D&bmatch=seng-v10-p-sm-isqm-2-o-0113",
"http://tinnhanh360.net/-ong-tay-de-thuong-ban-do-choi-dao-ha-noi-gay-ngac-nhien-thich-thu.html",
"http://metro.co.uk/2014/08/15/big-brother-2014-shock-twist-ashleigh-coyle-comes-second-4834723/",
"http://atwiki.jp/wiki/%EF%BC%AD%EF%BC%B5%EF%BC%A7%EF%BC%A5%EF%BC%AE%20%E6%82%9F%E7%A9%BA",
"http://emgn.com/tv/schnauzer-passes-out-with-excitement-when-he-is-reunited-with-his-human-after-2-years/",
"http://nguyentandung.org/bo-truong-nguyen-bac-son-se-manh-tay-ngan-chan-tin-nhan-rac.html",
"http://motthegioi.vn/tai-chinh-bat-dong-san/tu-17-nguoi-mua-nha-dat-phai-cong-them-hang-loat-chi-phi-187472.html",
"http://spankbang.com/4ze1/video/brunette+with+big+boobs+fucked+in+a+cellar+public+agent",
"http://torcache.net/torrent/FECDCFB88429EDAE6184E26D63EDD5A1AB2B1736.torrent?title=[kickass.to]bro.s.before.ho.s.2013.720p.brrip.nl.gesproken.dutchreleaseteam",
"http://mic.com/articles/117370/baltimore-reveals-why-we-need-our-musicians-now-more-than-ever",
"http://thenextweb.com/dd/2014/04/08/ux-designers-side-drawer-navigation-costing-half-user-engagement/feed/gtm.start",
"http://emgn.com/movies/person-of-interest-what-to-expect-from-season-4-release-date/",
"http://depositphotos.com/login.html?url=%2F34608623%2Fstock-illustration-vector-abstract-3d-road-infographics.html",
"http://serverfault.com/questions/52370/what-is-the-order-group-policy-settings-are-applied",
"http://kakaku.com/kaden/portable-dvd-player/ranking_2026/pricedown/div-gpt-ad-k/header_text",
"http://indianexpress.com/article/world/asia/pak-officer-gave-up-osama-kayani-isi-chief-helped-us/",
"http://nypost.com/2015/05/13/brooklyn-man-kills-cheating-girlfriends-lover-then-self/?share=email",
"http://distractify.com/post/related/id/5543af854a0c4b511d7c70a2/skip/30/limit/10/back/0",
"http://sourceforge.net/directory/communications/add_facet_filter?facet=os&constraint=Mac",
"http://babal.net/books/view/960/%D8%AE%D9%8A%D8%A7%D9%86%D8%A9-%D8%A7%D9%84%D9%85%D8%AB%D9%82%D9%81%D9%8A%D9%86",
"http://serverfault.com/questions/690852/use-powershell-to-start-a-gui-program-on-a-remote-machine",
"http://cookpad.com/login?rt=%2Fcategory%2Frecommend%2F590%3Frt%3D%252Fcategory%252F590",
"http://plarium.com/en/strategy-games/stormfall-age-of-war/news/withdraw-to-the-catacombs/",
"http://pikabu.ru/tag/%D1%81%D1%82%D0%B0%D0%B2%D1%80%D0%BE%D0%BF%D0%BE%D0%BB%D1%8C/hot",
"http://grantland.com/hollywood-prospectus/2015-oscar-nomination-predictions-best-director-best-picture/",
"http://allegro.pl/listing/user/listing.php?us_id=25662671&order=p&string=B-110+HIT&search_scope=userItems-25662671&bmatch=s0-default-0113&sh_dwh_token=d0b6dd74a8394345128e20d296cb1c5e",
"http://nguyentandung.org/bai-do-xe-ngam-trong-cong-vien-thong-nhat-thu-hoi-von-cach-nao.html",
"http://allegro.pl/listing/listing.php?bmatch=seng-v6-p-sm-isqm-2-ce-0113&id=175&order=m&string=%7Bstring%7D",
"http://europa.eu/about-eu/agencies/regulatory_agencies_bodies/policy_agencies/eurofound/index_en.htm",
"http://extratorrent.cc/torrent/4190380/VA-Unusual_Techno_One_LP-WEB-2015-CBR_INT.html",
"http://persianblog.ir/tags/42604/8/%d8%b3%d9%87%d8%b1%d8%a7%d8%a8_%d8%b3%d9%be%d9%87%d8%b1%db%8c/",
"http://distractify.com/post/related/id/5537c6314a0c4b89316cdbdf/skip/20/limit/10/back/0",
"http://kenh14.vn/star/lo-dien-ban-trai-hot-boy-cua-van-mai-huong-20140716121655295.chn",
"https://twitter.com/share?url=http%3A%2F%2Fhubpages.com%2Fhub%2FWhats-the-difference-between-self-esteem-and-confidence&text=Definitions+of+Confidence+%26+Self+Esteem+-+What%27s+the+Difference%3F",
"http://nesn.com/2015/04/draftkings-giancarlo-stanton-mookie-betts-in-tom-carons-fantasy-lineup/",
"http://fishki.net/1526766-kapello-zajavil-o-bezumii-prezidenta-rfs.html?mode=recent",
"http://syosetu.com/searchuser/search/index.php?name1st=%E3%82%80&all=1&all2=1&all3=1&all4=1&p=10",
"http://akhbarelyom.com/news/newdetails/411497/1/%D9%85%D8%B5%D8%B1-%D8%A7%D9%84%D8%B9%D8%B7%D8%A7%D8%A1-%D8%AA.html",
"http://allegro.pl/triumph-stringi-precious-essence-string-granat-40-i5035632976.html",
"http://getpocket.com/signup?mode=socialmode&route=http://getpocket.com/p/pocket-topics/fitness",
"http://mylust.com/videos/235751/sexy-hentai-chick-gets-her-huge-succulent-tits-squeezed/",
"http://censor.net.ua/photo_news/335629/v_tsentre_donetska_ural_boevikov_dnr_razdavil_jiguli_2_pogibshih_narod_ne_protestuet_foto",
"http://pornsharing.com/latina-girl-chloe-amour-and-busty-blonde-step-mom-julia-ann-get-face-fucked_v78082",
"http://genius.com/4810956/Ol-dirty-bastard-never-before-seen-ol-dirty-bastard-raw-and-uncut-interview/But-its-just-hey-the-kids-love-dirty-you-know-what-i-mean-every-time-they-see-dirty-they-smile-you-know-what-i-mean",
"http://sourceforge.net/directory/communications/add_facet_filter?facet=language&constraint=PL%2FSQL",
"http://codecanyon.net/item/bitdrop-file-hosting-with-short-url-link/full_screen_preview/426023",
"http://qz.com/404309/photos-silicon-valley-fashion-week-kicks-off-with-drone-models/",
"http://kenh14.vn/2-tek/ban-jailbreak-he-dieu-hanh-ios-613-toi-ios-615-chinh-thuc-trinh-lang-2013122404361169.chn",
"http://uproxx.com/movies/2015/04/paul-bettany-gives-us-a-glimpse-of-vision-in-action-in-avengers-age-of-ultron/",
"http://babal.net/women/view/80/%D8%AF%D9%84%D9%91%D9%84%D9%8A-%D9%86%D9%81%D8%B3%D9%83-%D8%A3%D9%8A%D8%AA%D9%87%D8%A7-%D8%A7%D9%84%D8%A3%D9%85-%D9%85%D8%B9-%D9%87%D8%B0%D9%87-%D8%A7%D9%84%D9%86%D8%B5%D8%A7%D8%A6%D8%AD",
"http://mic.com/articles/116370/avengers-stars-sexist-jokes-illustrate-a-major-problem-with-the-comic-book-industry",
"http://torcache.net/torrent/0FF0E2F39CD5AB403EE03F8E1B69127D0476D976.torrent?title=[kickass.to]komban.2015.hq.real.dvdscr.xvid.1cd.700mb.tamil",
"https://paytm.com/blog/paytm-offer-for-app-users-get-upto-rs-50-cash-back/?share=email",
"http://motthegioi.vn/suckhoe/nu-cuoi-y-hoc/nu-cuoi-suc-khoe-nhung-li-do-nen-yeu-chang-trai-bung-bu-188294.html",
"http://torcache.net/torrent/047D47DFF4DC5CD9BEA6D0F4C57D68F2F2D71205.torrent?title=[kickass.to]night.at.the.museum.secret.of.the.tomb.2014.1080p.brrip.x264.yify",
"http://thenextweb.com/asia/2014/09/26/myanmars-mobile-revolution-kicks-telenor-prepares-launch-service/gtm.start",
"http://extratorrent.cc/torrent/4189616/Jedi+Mind.Tricks.The.Thief.and.the.Fallen.2015.mp3.vbr.NOiR.html",
"http://genius.com/2990084/Us-general-services-administration-gsa-mentor-protege-program-subpart-51970/Incentives-for-prime-contractors",
"http://thenextweb.com/apps/2012/04/19/500px-launches-android-app-and-overhauls-its-ipad-version-too/",
"http://kienthuc.net.vn/tin-tuc-tuyen-sinh/ty-le-choi-cua-dai-hoc-fpt-nam-2014-487408.html",
"http://sourceforge.net/directory/development/add_facet_filter?facet=license&constraint=OSI-Approved+Open+Source+%3A%3A+PHP+License",
"http://nypost.com/2015/01/28/tepper-loses-fight-to-restructure-caesars-in-delaware/",
"http://ap.org/Content/Press-Release/2013/NFL-celebrates-season-with-NFL-Honors-Super-Bowl-Eve",
"http://indianexpress.com/article/cities/ludhiana/32-year-old-commits-suicide-at-in-laws-place-in-ludhiana/",
"http://extratorrent.cc/torrent_download/4191066/Chappie.2015.720p.WEB-DL.AAC2.0.H.264-PLAYNOW.torrent",
"http://correios.com.br/english/the-brazil-post/education-and-culture/cultural-centers/cultural-center-of-rio-de-janeiro",
"http://web.de/magazine/sport/fussball/champions-league/fc-bayern-muenchen-fc-barcelona/fc-bayern-muenchen-fc-barcelona-enrique-fuerchtet-freund-guardiola-30633596",
"http://noticias.uol.com.br/saude/album/2015/03/17/dengue-pelo-brasil.htm?abrefoto=63",
"http://bdnews24.com/economy/2015/05/05/world-bank-to-give-bangladesh-200-million-loan-to-cut-poverty",
"http://kenh14.vn/tv-show/viet-huong-gao-thet-to-chong-leng-pheng-trai-tre-20150208121726226.chn",
"http://codecanyon.net/item/photofans-your-social-network-to-share-photos/full_screen_preview/6308014",
"http://tunein.com/radio/Carolina-Mudcats-at-Myrtle-Beach-Pelicans-May-13-2015-p729095/",
"http://nguyentandung.org/vi-sao-nguyen-tong-bien-tap-bao-nguoi-cao-tuoi-bi-khoi-to.html",
"http://kickass.to/microsoft-office-professional-plus-2013-64-bit-english-rareabyss-t10091322.html",
"http://weathernews.jp/smart/sora_mission/0/glance.html?missionid=1431484304.67676&fm=sw&fmdotop=2",
"http://atwiki.jp/wiki/%E8%A9%A6%E9%A8%93%E4%B8%AD%20%E9%9C%87%E7%81%BD%E5%AF%BE%E5%BF%9C",
"http://plarium.com/en/strategy-games/sparta-war-of-empires/mmo-masters/offensive-positions",
"https://chaturbate.com/tipping/spy_on_private_show_tokens_per_minute/ingridblondy94/",
"http://babal.net/news/view/42903/%D8%AD%D8%A8%D8%B3-%D9%85%D8%AD%D8%A7%D9%81%D8%B8-%D8%A7%D9%84%D8%A3%D9%82%D8%B5%D8%B1-%D8%A7%D9%84%D8%B3%D8%A7%D8%A8%D9%82-%D8%B9%D8%A7%D9%85%D9%8A%D9%86-%D9%88%D8%AA%D8%BA%D8%B1%D9%8A%D9%85%D9%87-10-%D8%A2%D9%84%D8%A7%D9%81-%D8%AC%D9%86%D9%8A%D9%87-%D9%84%D8%B9%D8%AF%D9%85-%D8%AA%D9%86%D9%81%D9%8A%D8%B0%D9%87-%D8%AD%D9%83%D9%85%D8%A7-%D9%82%D8%B6%D8%A7%D8%A6%D9%8A%D8%A7",
"http://sberbank.ru/portalserver/sb-portal-ru/ru/person/paymentsandremittances/remittance/in/beznal_rus/beznal_snal_rus",
"http://torcache.net/torrent/3671994FE2E23CC3E914E74EB63E5A85AA184B4B.torrent?title=[kickass.to]focus.2015.720p.hq.ac3.dd5.1.externe.eng.nl.subs.tbs",
"http://torcache.net/torrent/110A54D90A4F523BF7051AAD5ADE8F5E72FC3ED2.torrent?title=[kickass.to]hentai.3d.milky.maid.may.2015",
"http://pikabu.ru/tag/%D0%90%D1%80%D0%B5%D0%BD%D0%B4%D0%B0%20%D0%B6%D0%B8%D0%BB%D1%8C%D1%8F/hot",
"http://mirtesen.ru/url?e=simple_click&blog_post_id=43032218893&url=http%3A%2F%2Fnarodnayamedicyna.ru%2Fblog%2F43032218893%2Fsudorogi",
"http://mirtesen.ru/url?e=simple_click&blog_post_id=43294960453&url=http%3A%2F%2Fs30469371943.mirtesen.ru%2Fblog%2F43294960453%2FAh-leto%21",
"http://distractify.com/post/related/id/54b985434a0c4b136ee6813d/skip/20/limit/10/back/0",
"http://jalopnik.com/5958144/gourmet-food-trucks-feed-storm-battered-and-starved-coney-island-residents",
"http://techcrunch.com/2015/01/11/from-a-rebuilt-60s-motel-in-vegas-orderwithme-tackles-big-e-commerce-challenges/",
"https://twitter.com/home?status=%E3%83%8C%E3%81%91%E3%82%8B%EF%BC%81%E3%80%90%E3%82%A2%E3%83%8B%E3%83%A1%E3%80%91+http%3A%2F%2Fero-video.net%2Ft%2FfnLTNjuNx6qWFt13+%E3%82%A2%E3%83%AB%E3%83%90%E3%82%A4%E3%83%88%E3%81%AE%E5%85%88%E8%BC%A9%E3%81%AE%E5%A5%B3%E3%81%AE%E3%82%B3%E3%81%A8%E3%82%A8%E3%83%83%E3%83%81%E3%80%80%E7%90%B4%E5%B9%B3%E3%81%95%E3%82%86+%23ero+%23douga+%23agesage",
"http://torcache.net/torrent/9CFDD892CDA9F5EEA5DB98A955E4E1382262E670.torrent?title=[kickass.to]arrow.s03e04.hdtv.x264.lol.eztv",
"http://thenextweb.com/insider/2014/10/29/unbabel-integrates-mailchimp-offer-translation-service-promotional-emails/gtm.start/",
"http://arstechnica.com/science/2015/05/carbon-emissions-put-sixteen-percent-of-species-at-risk-of-extinction/",
"http://tunein.com/radio/Economics-of-Political-Memoir---Marketplace-May-13-2015-p729173/15480783/ca-pub-1542925551861702/Program",
"http://torrentdn.com/bbs/s.php?bo_table=torrent_mid&wr_id=57742&k=%EC%99%95%EC%A2%8C&page=1",
"http://searchengineland.com/twitters-full-google-search-integration-is-coming-in-may-219941",
"http://nesn.com/2014/07/veteran-leaders-return-to-field-for-day-1-of-patriots-training-camp-video/",
"https://paytm.com/blog/pep-up-this-summer-season-with-paytms-first-ever-fashion-fest/",
"http://torcache.net/torrent/523EF7169F14C884FC2AD988451BBB6A4E0BD5F4.torrent?title=[kickass.to]project.almanac.2014.720p.brrip.x264.yify",
"http://metro.co.uk/2015/04/01/zayn-malik-loses-50000-twitter-follows-in-wake-of-one-direction-departure-but-dont-worry-hes-still-popular-5131044/",
"http://censor.net.ua/jokes/all/page/1/hero/0/category/138/interval/5/author/0/sortby/date",
"http://depositphotos.com/login.html?url=%2F25901805%2Fstock-photo-breakfast-with-coffee-orange-juice.html",
"http://thenextweb.com/eu/2015/05/09/meet-the-man-who-left-nasa-to-join-a-european-startup/gtm.start",
"http://putlocker.is/watch-night-at-the-museum-secret-of-the-tomb-online-free-putlocker.html",
"http://tunein.com/radio/Radio-Fredericton-905-s12073/15480783/ca-pub-1542925551861702/StationLeader",
"http://lifehacker.com/5910717/plan-your-free-online-education-at-lifehacker-u-summer-semester-2012",
"http://extratorrent.cc/torrent/4189607/The+Sleepwalker.2014.LiMiTED.DVDRip.x264.LPD.html",
"http://mylust.com/videos/64031/chinese-orgy-with-sexy-and-skinny-asian-babe-in-her-bedroom/",
"http://buzzfil.net/m/show-art/voici-16-voisins-qui-ont-pique-une-crise-de-nerfs-9.html",
"http://censor.net.ua/video_news/334975/ochered_za_pensieyi_v_okkupirovannom_luganske_ty_so_zvukom_zapishi_s_matyukami_video",
"http://ap.org/Content/Press-Release/2014/The-personal-news-cycle-how-Americans-get-their-news",
"http://akhbarelyom.com/news/newdetails/411484/1/%D9%85%D8%B5%D8%B1%D9%8A%D8%AA%D8%A7%D9%86-%D9%81%D9%82%D8%B7-.html",
"http://sourceforge.net/directory/communications/add_facet_filter?facet=os&constraint=Modern+%28Vendor-Supported%29+Desktop+Operating+Systems+%3A%3A+OpenBSD",
"http://kakaku.com/kaden/label-writer/ranking_2076/pricedown/div-gpt-ad-k/header_text",
"http://jezebel.com/jameis-winston-files-lawsuit-against-woman-who-accused-1703312090/+miamirohan",
"http://grantland.com/hollywood-prospectus/mad-men-power-rankings-episode-710-the-forecast/",
"https://www.gov.uk/government/world/uk-delegation-to-organization-for-security-and-co-operation-in-europe",
"http://emgn.com/entertainment/youll-be-left-speechless-at-the-wartime-bravery-of-these-22-animal-heroes/",
"http://bdnews24.com/politics/2015/05/02/dire-consequences-unless-city-polls-cancelled-bnp-s-hannan-shah",
"http://web.de/magazine/unterhaltung/adel/royal-baby-2/wohnt-prinzessin-charlotte-elizabeth-diana-30619870",
"http://extratorrent.cc/torrent/4189415/Snow%2C+Heather-Sweet+Deception+-+epub+-+zeke23.html",
"http://torcache.net/torrent/4EFE4986FC8B15951469CF6AC82FE102E8C9007A.torrent?title=[kickass.to]if.i.stay.2014.720p.brrip.x264.yify",
"https://twitter.com/share?url=http%3A%2F%2Fhubpages.com%2Fhub%2FEMT-Electrical-Conduit-Pipe-Bending-the-Math-Behind-a-Conduit-Bending-Guide&text=Formulas+and+Multipliers+For+Bending+Conduit+or+Electrical+Pipe",
"http://seekingalpha.com/article/2471615-mexico-economy-to-move-with-energy-reforms-part-1",
"http://web.tv/liveCategory/6/language/1/index/changeLanguage/newshared/searchAutoComplete",
"http://indianexpress.com/article/sports/football/chelsea-go-past-leicester-city-3-1-eye-premier-league-title/",
"http://web.de/magazine/sport/fussball/champions-league/wunder-fc-schalke-04-real-madrid-champions-league-17321480",
"http://olx.co.id/i2/elektronik-gadget/komputer/keyboard-mouse/elektronik-gadget/komputer/keyboard-mouse",
"http://mylust.com/videos/171589/my-lewd-chinese-wife-is-really-into-having-sex-in-missionary-position/",
"http://babal.net/books/view/397/%D9%82%D8%A7%D9%86%D9%88%D9%86-%D8%BA%D8%B1%D9%81-%D8%A7%D9%84%D8%B5%D9%86%D8%A7%D8%B9%D8%A9-%D8%A7%D9%84%D8%A7%D8%B1%D8%AF%D9%86%D9%8A",
"http://techcrunch.com/2014/03/18/hurry-this-is-your-last-chance-to-get-tickets-to-tonights-washington-d-c-meetup/",
"http://distractify.com/post/related/id/54b985434a0c4b136ee6813d/skip/30/limit/10/back/0",
"http://kakaku.com/daily-goods/antimold-agent/ranking_7633/pricedown/div-gpt-ad-k/header_text",
"http://metro.co.uk/2015/04/30/harry-styles-spotted-minding-his-own-busines-son-los-angeles-bus-5174286/",
"http://kienthuc.net.vn/tin-tuc-tuyen-sinh/ung-vien-du-hoc-theo-de-an-599-cp-huong-hoc-phi-400-trieunam-486551.html",
"http://web.de/magazine/reise/blog/gundel-woite/feucht-froehlicher-zwischenfall-donegal-30566558",
"http://thenextweb.com/insider/2012/10/30/as-prezi-amasses-more-than-15-million-users-it-promotes-the-idea-economy-and-moves-into-the-enterprise/gtm.js",
"https://www.gov.uk/government/policies/economic-development-in-coastal-and-seaside-areas",
"http://olx.ua/uk/list/q-%D0%B1%D0%B0%D0%BB%D0%B5%D1%82%D0%BA%D0%B8/bIE64RPKp0urlDE3vSbHw8dRLeUY1kOFK2_KOEmw9UT.A7",
"http://olx.pl/uslugi-firmy/kategorie/JOcK7gzdoyrcIT2176iLzhCz8UoPOGcT6Y1ax8FNTPf.l7",
"http://babal.net/photos/54/%D8%AE%D9%84%D9%81%D9%8A%D8%A7%D8%AA-%D9%88%D9%8A%D9%86%D8%AF%D9%88%D8%B2",
"http://correios.com.br/Para-governo/governo-municipal/solucoes-gratuitas-de-apoio-e-gestao/sigep-web",
"https://myspace.com/article/2015/4/27/family-amy-winehouse-says-amy-documentary-misleading",
"http://indianexpress.com/article/good-news/covered-in-tar-street-dog-rescued-in-three-hours/",
"http://web.de/magazine/gesundheit/blog/ulli-goebl/7-nahrungsmittel-stimmung-energielevel-jahr-heben-30335162",
"http://olx.in/hi/item/nokia-lumia-520-with-3-back-covers-and-data-cable-and-box-IDWjzgb.html",
"http://squarespace.com/press/2015/1/7/squarespace-announces-a-major-upgrade-to-its-platform-with-new-getty-images-and-google-integrations",
"https://privatbank.ua/ua/business/otraslevyje-reshenija-dlya-biznesa/resheniya-agrobiznesu/",
"http://distractify.com/post/related/id/553017ed4a0c4b853fa28e2e/skip/10/limit/10/back/0",
"http://tunein.com/radio/The-Ian-OConnor-Show-p335837/15480783/ca-pub-1542925551861702/Program",
"http://thenextweb.com/apps/2011/07/24/the-complete-list-of-top-instagram-apps/carousel/",
"http://torcache.net/torrent/38C97B7056A1B4E72811C617C1DB6D03B081A533.torrent?title=[kickass.to]the.blacklist.s02e12.the.kenyon.family.x264.dtw",
"http://distractify.com/post/related/id/554ba22e4a0c4b793e066a77/skip/20/limit/10/back/0",
"http://extratorrent.cc/torrent_download/4189576/Game+Of+Thrones+-+Il+Trono+Di+Spade+S05e04%5BMux+-+720p+-+H264+-+Ita+Eng+Ac3+-+Sub+Ita+Eng%5D+DLMux+-+GiuseppeTnT.torrent",
"http://xhamster.com/movies/4567879/you_are_going_to_love_the_way_that_stockings_feel.html",
"http://perezhilton.com/2015-05-13-student-detention-dress-code-violation-letter-suspension-gender-sexist/?from=blogroll_perezhilton",
"http://comicbook.com/2015/01/20/chris-evans-and-chris-pratt-exchange-barbs-in-heated-super-bowl-/",
"https://paytm.com/offer/wp-content/plugins/wc-gallery/includes/js/flexslider/flexslider.css?ver=2.2.0",
"http://grantland.com/hollywood-prospectus/we-went-there-celebrating-25-years-of-goodfellas-with-bobby-d-ray-paulie-and-lorraine-at-tribeca-film-festival/",
"http://distractify.com/post/related/id/55479c6a4a0c4bc56b941a7e/skip/10/limit/10/back/0",
"http://thenextweb.com/apple/2015/05/10/11-things-i-learned-during-two-weeks-with-an-apple-watch/gtm.start",
"http://icicibank.com/Personal-Banking/cards/debit-card/debit-cards/the-gemstone-collection.page",
"http://deadspin.com/5746724/is-erin-andrews-reebok-endorsement-deal-journalistically-unethical",
"http://udn.com/news/story/7341/893933-%E7%8E%8B%E7%A5%96%E5%A3%BD%EF%BC%8F%E9%84%A7%E9%BA%97%E5%90%9B%E3%80%8C%E5%BE%A9%E7%94%9F%E3%80%8D-%E8%B2%BB%E7%8E%89%E6%B8%85%E3%80%8C%E5%BE%A9%E5%87%BA%E3%80%8D",
"http://bestblackhatforum.com/Forum-Announcements-And-Updates&datecut=0&sortby=lastpost&order=asc",
"https://quizlet.com/login?redir=https%3A%2F%2Fquizlet.com%2Flogin%3Fredir%3Dhttps%253A%252F%252Fquizlet.com%252Fcreate-set%252Fnew",
"http://depositphotos.com/60963767/stock-photo-design-team-working-together-while.html",
"http://pitchfork.com/features/profiles/9646-love-is-strange-the-multitudes-of-unknown-mortal-orchestras-ruban-nielson/",
"http://emgn.com/entertainment/coldplay-announce-release-date-for-6th-album-ghost-stories/",
"http://katproxy.com/fast-and-furious-7-2015-hd-ts-xvid-ac3-hq-hive-cm8-t10472303.html",
"http://elitedaily.com/sports/marcus-mariota-set-become-first-hawaiian-nfl-superstar/1015927/",
"http://extratorrent.cc/torrent_download/4191159/The+Longest+Yard+%282005%29+720p+WEB-DL+900MB+-+MkvCage.torrent",
"http://pornsharing.com/handcuffed-prisoner-gets-his-big-black-dick-sucked-by-angelik-duval_v72019",
"http://thenextweb.com/apps/2011/07/24/the-complete-list-of-top-instagram-apps/webstagram/gtm.start",
"http://distractify.com/post/related/id/5553b0104a0c4b621246c516/skip/50/limit/10/back/0",
"http://atwiki.jp/wiki/%E3%82%AA%E3%83%96%E3%83%AA%E3%83%93%E3%82%AA%E3%83%B3%20%E9%A1%94",
"http://codecanyon.net/item/phpforum-v15-social-forum-script/full_screen_preview/11340982",
"http://interpark.com/product/MallDisplay.do?_method=Detail&sc.shopNo=0000100000&sc.dispNo=001930&sc.prdNo=26600264&bm1=mid&bm2=hotsale_prd",
"http://gtbank.com/personalbanking/cards/mastercard/gtbank-dollar-credit-cards/gtbank-world-mastercard",
"https://medium.com/@theemilyhopkins/another-bad-commencement-address-90758e67ab08?source=latest",
"http://torcache.net/torrent/F52DF00E79B66A462A1848B6478A15D8EAD88679.torrent?title=[kickass.to]the.walking.dead.seizoen3.afl.10.hdtv.xvid.nl.subs.dmt",
"http://correios.com.br/english/the-brazil-post/history-and-facts/mailmen-in-brazil/imagens/CarteirosdoBrasil_1.jpg/@@images/652365fb-0395-469a-9812-e75aea0d1562.jpeg",
]

with ThreadPoolExecutor() as executor:
    executor.map(validate_existence, list_domain)
# validate_existence(url)


Domain http://tinnhanh360.net/-ong-tay-de-thuong-ban-do-choi-dao-ha-noi-gay-ngac-nhien-thich-thu.html [---]
Domain http://emgn.com/tv/schnauzer-passes-out-with-excitement-when-he-is-reunited-with-his-human-after-2-years/ [---]
Domain http://motthegioi.vn/tai-chinh-bat-dong-san/tu-17-nguoi-mua-nha-dat-phai-cong-them-hang-loat-chi-phi-187472.html [---]
Domain http://emgn.com/movies/person-of-interest-what-to-expect-from-season-4-release-date/ [---]
Domain http://pikabu.ru/tag/%D1%81%D1%82%D0%B0%D0%B2%D1%80%D0%BE%D0%BF%D0%BE%D0%BB%D1%8C/hot [---]
Domain http://motthegioi.vn/suckhoe/nu-cuoi-y-hoc/nu-cuoi-suc-khoe-nhung-li-do-nen-yeu-chang-trai-bung-bu-188294.html [---]
Domain http://correios.com.br/english/the-brazil-post/education-and-culture/cultural-centers/cultural-center-of-rio-de-janeiro [---]
Domain http://sberbank.ru/portalserver/sb-portal-ru/ru/person/paymentsandremittances/remittance/in/beznal_rus/beznal_snal_rus [---]
Domain http://pikabu.ru/tag/%D0%90%D1%80%D0%B5%D0%BD%D0%B4%D0